In [1]:
import matplotlib.pyplot as plt
from sklearn import preprocessing
import numpy as np
import pandas as pd
import warnings
from scipy import stats
from sklearn.feature_selection import RFECV
from scipy import interpolate
from pandas import DataFrame
#import seaborn as sns
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import random
from sklearn import metrics
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from datetime import datetime, timedelta
import math
from hmmlearn import hmm

In [14]:
col_Names=["id", "imo", "lat", "lon", "course", "heading", "speed", "timestamp", "name", "type_name", "destination"]

df=pd.read_csv("C:/Users/Χαρά/Desktop/generalCargo.csv", names = col_Names)
null_ids = df['id'].isnull().values.any() # check if any id is null
null_ids

False

In [15]:
m = len(df)
print(m)

2453488


In [16]:
null_heading = df['heading'].isnull().values.any() # check if any heading is null
null_heading # true or false

True

In [17]:
#values = {'heading': 'Unknown Heading', 'destination': 'Unknown Destination'}
values = {'destination': 'Unknown Destination'}
df = df.fillna(value=values) # fill nans of only the heading column's and destination column's

In [18]:

# let's now generate some random numbers to specify what label from the enumeration the logic chose to 
# put as label next to a point the number of its appearance will also be randomly specified by
# these 2 below functions...

frequency = np.random.choice(np.arange(1, 7))
print(frequency)

label = np.random.choice( ['A', 'B', 'C', 'D', 'E', 'F', 'G'] )
print(label)

# let's find out how many nan values inside navigation column we have at this point left empty so we generate as many 
# letter labels as we need ...

NaN = np.nan

df['navigation'] = NaN
print(len(df['navigation']))



letters = [letter for i in range(len(df.loc[df.navigation.isnull(), 'navigation']))
                for letter in np.random.choice( ['A', 'B', 'C', 'D', 'E', 'F', 'G'] )
                   for frequency in range(np.random.choice(np.arange(1, 5)))]




2
C
2453488


In [20]:
print(len(letters))
print(len(df['navigation']))
letters_trimed = letters[:2453488]


# fill the rest values which are nan from the navigation column since we didn't find out all real labels 
# with values from letters list/series which was generated above with a random frequency of appearance of each randomly chosen letter
# to do so we need to have that list named letters the same length as our nans are inside our navigation column
# let's also note that the order of the sequence won't change! the letters will fill the nan values following the exact same 
# order that they have inside the 'letters' list...
df.loc[df.navigation.isnull(), 'navigation'] = letters_trimed

6132754
2453488


In [21]:
df

,id,imo,lat,lon,course,heading,speed,timestamp,name,type_name,destination,navigation
0,256746000,9327774,36.97554,12.57211,279,279.0,113,2015-08-01 00:00:00,SPAVALDA,General Cargo,GIBRALTAR,G
1,241374000,7903249,38.00850,16.62183,217,511.0,101,2015-08-01 00:00:00,EVIACEMENT IV,General Cargo,LICATA,G
2,671240000,8812875,36.58783,23.52683,224,247.0,90,2015-08-01 00:00:00,HUDA F,General Cargo,Y,E
3,244017000,9246906,37.58095,15.88839,210,210.0,67,2015-08-01 00:00:00,FRISIAN LADY,General Cargo,MALTA,B
4,271040283,9473975,36.22883,15.73128,283,282.0,86,2015-08-01 00:00:00,ATLAS B,General Cargo,CASTELLON,B
5,636016745,9506588,35.71525,14.56147,245,245.0,103,2015-08-01 00:00:01,BEE,General Cargo,GABES/TUNISIAN,B
6,538005972,9268083,36.30018,14.46795,269,268.0,114,2015-08-01 00:00:01,AUCKLAND SPIRIT,General Cargo,GIBRALTAR,B
7,271000261,8913320,40.96595,28.93332,48,345.0,0,2015-08-01 00:00:01,ZINNET METE,General Cargo,YUZHNY,B
8,374749000,8516744,40.87383,29.18370,66,31.0,0,2015-08-01 00:00:02,LIRA,General Cargo,KARTAL,B
9,377768000,8130851,41.88612,28.86559,163,161.0,108,2015-08-01 00:00:03,SITHONIA,General Cargo,BALCHIK,F


In [26]:
# save the new dataframe into a .csv file...
df.to_csv(r'C:\Users\Χαρά\Desktop\final.csv', index=False, header=False)